In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Just essential tools

In [8]:
# df_5 is a 5-minute dataframe, where each row is a 5-minute candle
# Same applies to df_1, but it's recent data and shorter
df_5 = pd.read_pickle('data/KAS_USDT_5m_300_days_data.pkl')
df_1 = pd.read_pickle('data/KAS_USDT_1m_10_days_data.pkl')

df_5.head(3)

,open,close,high,low,volume
timestamp,,,,,
2024-01-05 11:55:00,0.112729,0.113025,0.113134,0.112401,67258.53
2024-01-05 12:00:00,0.113025,0.113068,0.113529,0.11283,132406.52
2024-01-05 12:05:00,0.113068,0.1119,0.113068,0.111899,196303.66


In [11]:
df_5.describe()

,open,close,high,low,volume
count,17300,17300,17300,17300,17300
unique,14605,14607,13707,13709,17296
top,0.17,0.17,0.17,0.165,0
freq,29,29,50,18,4
